In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from matplotlib.patches import Patch, Circle, PathPatch
import mpl_toolkits.mplot3d.art3d as art3d
import os 
from sklearn.metrics import mean_squared_error

rel_dir = '/home/kitadam/ENR_Sven/musings/data/jet-all-full.csv'
jet_pdb_all = pd.read_csv(rel_dir)
jet_pdb_all = jet_pdb_all.convert_dtypes()
jet_pdb = jet_pdb_all[(jet_pdb_all['elongation'] != -1) & (jet_pdb_all['Zeff'] != -1)]
jet_pdb['P_NBI(MW)'][jet_pdb['P_NBI(MW)'] < 0] = 0.0 
jet_pdb['gasflowrateofmainspecies10^22(e/s)'][jet_pdb['gasflowrateofmainspecies10^22(e/s)'] < 0] = 0.0 
jet_pdb.loc[:, 'divertorconfiguration'] = jet_pdb.loc[:, 'divertorconfiguration'].astype('category')
jet_pdb.loc[:, 'divertorconfiguration'] = jet_pdb.loc[:, 'divertorconfiguration'].cat.codes
jet_pdb['wall'] = [int(i) for i in (jet_pdb['shot'] < 80000).to_list()]
global_cols = ['BetaN(MHD)', 'Zeff']
info_cols = ['shot', 't1', 't2']
ped_cols = ['nepedheight10^19(m^-3)', 'error_nepedheight10^19(m^-3)']
mp_cols = ['Ip(MA)', 'B(T)', 'a(m)', 'q95','averagetriangularity', 'plasmavolume(m^3)','elongation','P_NBI(MW)', 'P_ICRH(MW)', 'P_TOT=PNBI+Pohm+PICRH-Pshi(MW)',   'gasflowrateofmainspecies10^22(e/s)', ]
cat_cols = ['FLAG:Kicks', 'FLAG:RMP', 'FLAG:pellets', 'divertorconfiguration', 'Atomicnumberofseededimpurity']
flags = ['FLAG:DEUTERIUM', 'FLAG:HYDROGEN', 'FLAG:H/Dmix', 'FLAG:HeJET-C', 'FLAG:Seeding', 'FLAG:Kicks', 'FLAG:RMP', 'FLAG:pellets', 'FLAG:HRTSdatavalidated', 'divertorconfiguration', 'Atomicnumberofseededimpurity',]
os.environ['PLOTSTYLE'] = '/home/kitadam/ENR_Sven/basematplotlibstyle.mplstyle'
if os.getenv('PLOTSTYLE') is not None: 
    plt.style.use(os.getenv('PLOTSTYLE'))
RED = "#dd3015"
GREEN = "#489A8C"
DARK = "#1C2C22"
GOLD = "#F87D16"
WHITE = "#FFFFFF"
BLUE = "#2E6C96"
PURPLE = '#d7a7ef'
LIGHTBLUE = "#a7bfef"

In [ ]:
jet_c = jet_pdb[jet_pdb['shot'] < 81000]
jet_ilw = jet_pdb[jet_pdb['shot'] > 81000]
lorenzo_inputs = ['Ip(MA)', 'averagetriangularity', 'P_NBI(MW)','Meff']
inputs = ['Ip(MA)', 'B(T)', 'a(m)', 'P_NBI(MW)', 'P_ICRH(MW)', 'P_TOT=PNBI+Pohm+PICRH-Pshi(MW)', 'plasmavolume(m^3)', 'q95', 'gasflowrateofmainspecies10^22(e/s)', 'elongation', 'averagetriangularity', 'FLAG:Kicks', 'FLAG:RMP', 'FLAG:pellets', 'Atomicnumberofseededimpurity', 'divertorconfiguration', 'wall']

targets = 'nepedheight10^19(m^-3)'

With high dimensional data, when splitting the data into train-test subsets, it is very unlikely that the test set falls into the convex hull of the training set, thus by measuring performance on the test set, we are likely measuring performance of the extrapolation rather than interpolation capabilities of the model [@HighDimLearning] (described for dummies [here](https://lauraruis.github.io/2021/11/06/extra.html)). 

The dilema for me is if we are always extrapolating, then somehow the UQ for a model should be defined in terms of the distance from the convex hull. Therefore, I want to investigate the properties of error in prediction for various models w.r.t the distance from the quiered point to the convex hull of the training set.  

## Subset in convex hull? 

First we need an algorithm to see if a point is within the convex hull of the training set. 

- Maths 
- Algorithm as a linear programming problem [Stackoverflow Implementation](https://stackoverflow.com/questions/16750618/whats-an-efficient-way-to-find-if-a-point-lies-in-the-convex-hull-of-a-point-cl)

We can check this then for the JET Pedestal database for varying input dimensions and splits. 

- **NB** Is the convex hull defined by the point cloud of input variables as well as target variable? Yes probably. 


In [ ]:
import random 
from scipy.optimize import linprog

random.seed(42)
shuffled_idxs = random.sample(jet_pdb.index.to_list(), k=len(jet_pdb))

jet_train, jet_test = jet_pdb.loc[shuffled_idxs[:-200]], jet_pdb.loc[shuffled_idxs[-200:]]

X_train, y_train = jet_train[inputs], jet_train[targets]
X_test, y_test = jet_test[inputs], jet_test[targets]
# X_all, y_all = jet_pdb[inputs], jet_pdb[targets]

def in_hull(points, x): # 
  n_points = len(points)
  n_dim = len(x)
  c = np.zeros(n_points)
  A = np.r_[points.T,np.ones((1,n_points))]
  b = np.r_[x, np.ones(1)]
  lp = linprog(c, A_eq=A, b_eq=b)
  return lp.success


plt.scatter(X_train[inputs[0]], X_train[inputs[1]])


for i in range(len(X_test)): 
  if in_hull(X_train.values, X_test.values[i]): 
    plt.scatter(X_test.values[i, 0], X_test.values[i, 1], color=RED)
  else: 
    plt.scatter(X_test.values[i, 0], X_test.values[i, 1], color=PURPLE)

plt.show()

## Distance to convex hull? 

This is probably tricky, as we would have to calculate the convex hull for a high dimensional space. 

Maybe we should instead quantify the distance of the point to the closest point __inside__ the convex hull? 

## Applications 

1. Could we make graphs of the error vs distance vs dimenisonality of input space for both the parameter space as well as latent space for DIVA?
2. How does error vs distance to hull vary as a function of model? And input space? 